In [16]:
import jsonlines
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import tqdm
import random
import time
import pickle
import re

In [2]:
name_songkick_url_pairs = []
with jsonlines.open('../data_for_558_proj/songkick.jl') as reader:
    for obj in reader:
        name_songkick_url_pairs.append((obj["name"], obj["url"]))

In [3]:
part_1 = name_songkick_url_pairs[:len(name_songkick_url_pairs)//2]
part_2 = name_songkick_url_pairs[len(name_songkick_url_pairs)//2:]

In [4]:
len(part_1), len(part_2)

(2793, 2794)

In [5]:
with open("songkick_names_part1.p", "wb") as f:
    pickle.dump(part_1, f)

with open("songkick_names_part2.p", "wb") as f:
    pickle.dump(part_2, f)

In [6]:
del name_songkick_url_pairs
del part_2

In [14]:
def get_text(html_object):
    if "text" in html_object:
        return html_object["text"].strip()
    else:
        return html_object.get_text().strip()

def get_link(html_object):
    if "href" in html_object.attrs:
        return html_object.attrs["href"]
    return ""

def clean_name_of_footnote(text):
    return re.sub('\[\d+\]', '', text)

In [8]:
occupation_triggers = ["musician", "rapper", "vocalist", "singer", "songwriter",
                  "record producer", "guitarist", "pianist", "cellist", "violinist",
                  "bassist", "drummer", "dj", "remixer", "saxophonist", "trumpet",
                  "clarinetist", "trombonist", "popstar", "rocker", "songwriter", "band"]

genre_triggers = []

with open("musicbrainz_genres.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        genre_triggers.append(row[0].lower())

In [9]:
search_query = """
https://en.wikipedia.org/w/api.php?action=query&format=json&list=search&srsearch=
"""
base_query = "https://en.wikipedia.org/wiki/"

In [10]:
part_1[118]

('Snow Tha Product',
 'https://www.songkick.com/artists/4078461-snow-tha-product')

In [19]:
found_search = []
dictionaries = []
s = requests.Session()
a = requests.adapters.HTTPAdapter(max_retries=5)
s.mount('http://', a)
s.mount('https://', a)
# previous_run_count = 1667
# part_1 = part_1[previous_run_count:]

for i, name_url in enumerate(tqdm.tqdm(part_1)):
    if i % 100 == 0 and i > 0:
        sleep_seconds = random.randint(1, 20)
        print("Sleeping for {}".format(sleep_seconds))
        time.sleep(sleep_seconds)
        print("done sleeping")

    name = name_url[0]
    url = name_url[1]
    dict_repr = {"name" : name, "songkick_url" : url}
    title = ""
    x = s.get(search_query+name, timeout=(3.05, 18.05))
    possible_results = x.json()["query"]["search"]
    for result in possible_results:
        r_title = result["title"]
        lower_case_title = r_title.lower()
        for occupation in occupation_triggers:
            if occupation in lower_case_title:
                title = r_title
                break
        if len(title) > 0:
            break
        snippet = result["snippet"].lower()
        for occupation in occupation_triggers:
            if occupation in snippet:
                title = r_title
                break
        if len(title) > 0:
            break
        for genre in genre_triggers:
            if genre in snippet:
                title = r_title
                break

    if len(title) > 0:
        wiki_url = base_query+title
        y = s.get(wiki_url, timeout=(3.05, 18.05))
        dict_repr["wiki_title"] = title
        dict_repr["wiki_url"] = wiki_url
        found_search.append(1)
    else:
        found_search.append(0)
        dictionaries.append(dict_repr)
        continue
    
    status_code = y.status_code
    
    if status_code == 200:
        soup_html = BeautifulSoup(y.text, 'lxml')
        info_table = soup_html.find("table", {"class": "infobox"})
        if info_table != None:
            rows = info_table.findAll("tr")
            for row in rows:
                pos_header = row.find("th")
                pos_data = row.find("td")
                if pos_header != None:
                    if pos_data != None:
                        header_text = get_text(pos_header)
                        data_text = get_text(pos_data)
                        row_links = pos_data.findAll("a")
                        links = []
                        for link in row_links:
                            link_text = clean_name_of_footnote(get_text(link))
                            if len(link_text) > 0:
                                link_link = get_link(link)
                                links.append({"text" : link_text, "link" : link_link})
                        dict_repr[header_text] = {"text" : data_text, "links" : links}
            dictionaries.append(dict_repr)
        else:
            dict_repr["table_not_found"] = 1
            dictionaries.append(dict_repr)
    else:
        print(status_code)
        dict_repr["status_code"] = status_code
        dictionaries.append(dict_repr)
    
    

  4%|▎         | 100/2793 [01:07<35:39,  1.26it/s]

Sleeping for 14
done sleeping


  7%|▋         | 200/2793 [02:35<1:06:49,  1.55s/it]

Sleeping for 13
done sleeping


 11%|█         | 300/2793 [03:47<21:38,  1.92it/s]  

...

 93%|█████████▎| 2600/2793 [32:04<02:23,  1.35it/s]

Sleeping for 17
done sleeping


 97%|█████████▋| 2700/2793 [33:30<01:01,  1.50it/s]

Sleeping for 11
done sleeping


100%|██████████| 2793/2793 [34:44<00:00,  1.34it/s]


In [15]:
len(dictionaries)

1667

In [21]:
with open("part_1.p", "wb") as f:
    data = {"found_search" : found_search, 
            "dictionaries" : dictionaries, 
            "count" : len(dictionaries)}
    pickle.dump(data, f)

In [20]:
dictionaries[0]

{'name': 'The Beach Boys',
 'songkick_url': 'https://www.songkick.com/artists/268425-beach-boys',
 'wiki_title': 'The Beach Boys',
 'wiki_url': 'https://en.wikipedia.org/wiki/The Beach Boys',
 'Origin': {'text': 'Hawthorne, California, U.S.',
  'links': [{'text': 'Hawthorne, California',
    'link': '/wiki/Hawthorne,_California'}]},
 'Genres': {'text': 'Rock\npop\nsurf\npsychedelia',
  'links': [{'text': 'Rock', 'link': '/wiki/Rock_music'},
   {'text': 'pop', 'link': '/wiki/Pop_music'},
   {'text': 'surf', 'link': '/wiki/Surf_music'},
   {'text': 'psychedelia', 'link': '/wiki/Psychedelic_music'}]},
 'Years active': {'text': '1961–present', 'links': []},
 'Labels': {'text': 'Candix\nCapitol\nBrother\nReprise\nCaribou\nCBS',
  'links': [{'text': 'Candix', 'link': '/wiki/Candix_Records'},
   {'text': 'Capitol', 'link': '/wiki/Capitol_Records'},
   {'text': 'Brother', 'link': '/wiki/Brother_Records'},
   {'text': 'Reprise', 'link': '/wiki/Reprise_Records'},
   {'text': 'Caribou', 'link': '